# Creating figures for data from a single group

## Required data 

Each participant should have a separate folder where their spectra and MRS voxels should be stored. The folders should be arranged as follows:

```
project_folder
|   participants.tsv
|
└───data_folder
    |    
    └───sub-01
    |   |    sub-01_spectrum.tsv
    |   |    sub-01_spectrum_frequencies.tsv
    |   |    sub-01_mrs-voxel_mni-space.nii.gz
    |   |    ...
    |
    └───sub-02
    |   |    sub-02_spectrum.tsv
    |   |    sub-02_spectrum_frequencies.tsv
    |   |    sub-02_mrs-voxel_mni-space.nii.gz
    |   |    ...
    |
    |...
```

Participant IDs should be contained in the participants.tsv file. They should be in a column entitled "participant_id".  

Spectrum files should contain a single column with the preprocessed MRS spectra that you wish to display. The related frequencies file should contain a single column with the corresponding frequencies in PPM. 

In this example we use a MEGA-PRESS difference spectra created with the Gannet toolbox (http://www.gabamrs.com/). In the case of Gannet, these files must be written out from the data file created during the analysis. This step will differ depending on the analysis tools that you are using. 

MRS voxels should be in compressed NIFTI format. These should have been aligned to the MNI152 standard space with a 2mm resolution. 


## Make a voxel overlap image

First define the directories and files to be used and load in the participant IDs.

In [ ]:
import os
import numpy as np
import pandas as pd

# Project directory
data_dir = '~/work/MRS-voxel-plot/how-to/'

# Directory to create figures in
fig_dir = os.path.join(data_dir,'figures')

# Check if this figure directory exists, and if not create it
if not os.path.isdir(fig_dir):
  os.mkdir(fig_dir)

